In [1]:
from docplex.mp.model import Model
from docplex.util.environment import get_environment
from collections import namedtuple
import pandas as pd

In [3]:
#Village
ElecDemand=pd.read_csv(r'C:\Users\research\Dropbox\PhD\Data\Zernez\Basic_GIS_shapefiles\VillageElecDemand.csv')
HeatDemand=pd.read_csv(r'C:\Users\research\Dropbox\PhD\Data\Zernez\Basic_GIS_shapefiles\VillageHeatDemand.csv')
TotalRoofArea=pd.read_csv(r'C:\Users\research\Dropbox\PhD\Data\Zernez\Basic_GIS_shapefiles\VillageRoofArea.csv')
Hydro=pd.read_excel(r'C:\Users\research\Dropbox\PhD\Publications\ECOS\RenewableZernez.xlsx','hydro',parse_cols='A')
DGSHP=pd.read_csv(r'C:\Users\research\Dropbox\PhD\Data\Zernez\Basic_GIS_shapefiles\VillageGSHP.csv')
totalPV=pd.read_csv(r'C:\Users\research\Dropbox\PhD\Data\Zernez\Basic_GIS_shapefiles\VillagePV.csv')
totalST=pd.read_csv(r'C:\Users\research\Dropbox\PhD\Data\Zernez\Basic_GIS_shapefiles\VillageST.csv')
Temp=pd.read_excel(r'E:\docplex-examples-master\Temp.xlsx','Temp',parse_cols='A:B') #specific to zernez
#Econstraint=pd.read_csv(r'E:\Master\Village\EconstraintVillage.csv')
#Econstraint

In [40]:
#Neighbourhood
ElecDemand=pd.read_csv(r'C:\Users\research\Dropbox\PhD\Data\Zernez\Basic_GIS_shapefiles\NeighbourhoodElecDemand.csv')
HeatDemand=pd.read_csv(r'C:\Users\research\Dropbox\PhD\Data\Zernez\Basic_GIS_shapefiles\NeighbourhoodHeatDemand.csv')
TotalRoofArea=pd.read_csv(r'C:\Users\research\Dropbox\PhD\Data\Zernez\Basic_GIS_shapefiles\NeighbourhoodRoofArea.csv')
Hydro=pd.read_excel(r'C:\Users\research\Dropbox\PhD\Publications\ECOS\RenewableZernez.xlsx','hydro',parse_cols='A')
DGSHP=pd.read_csv(r'C:\Users\research\Dropbox\PhD\Data\Zernez\Basic_GIS_shapefiles\NeighbourhoodGSHP.csv')
totalPV=pd.read_csv(r'C:\Users\research\Dropbox\PhD\Data\Zernez\Basic_GIS_shapefiles\NeighbourhoodPV.csv')
totalST=pd.read_csv(r'C:\Users\research\Dropbox\PhD\Data\Zernez\Basic_GIS_shapefiles\NeighbourhoodST.csv')
Temp=pd.read_excel(r'E:\docplex-examples-master\Temp.xlsx','Temp',parse_cols='A:B') #specific to zernez
#Econstraint=pd.read_csv(r'E:\Master\Village\EconstraintVillage.csv')
#Econstraint

In [ ]:
#District
ElecDemand=pd.read_csv(r'C:\Users\research\Dropbox\PhD\Data\Zernez\Basic_GIS_shapefiles\DistrictElecDemand.csv')
HeatDemand=pd.read_csv(r'C:\Users\research\Dropbox\PhD\Data\Zernez\Basic_GIS_shapefiles\DistrictHeatDemand.csv')
TotalRoofArea=pd.read_csv(r'C:\Users\research\Dropbox\PhD\Data\Zernez\Basic_GIS_shapefiles\DistrictRoofArea.csv')
Hydro=pd.read_excel(r'C:\Users\research\Dropbox\PhD\Publications\ECOS\RenewableZernez.xlsx','hydro',parse_cols='A')
DGSHP=pd.read_csv(r'C:\Users\research\Dropbox\PhD\Data\Zernez\Basic_GIS_shapefiles\DistrictGSHP.csv')
totalPV=pd.read_csv(r'C:\Users\research\Dropbox\PhD\Data\Zernez\Basic_GIS_shapefiles\DistrictPV.csv')
totalST=pd.read_csv(r'C:\Users\research\Dropbox\PhD\Data\Zernez\Basic_GIS_shapefiles\DistrictST.csv')
Temp=pd.read_excel(r'E:\docplex-examples-master\Temp.xlsx','Temp',parse_cols='A:B') #specific to zernez
#Econstraint=pd.read_csv(r'E:\Master\Village\EconstraintVillage.csv')
#Econstraint

In [43]:
def load_data(Group,ElecDemand,HeatDemand,totalPV,totalST,RoofArea,DGSHP):
    ZerDemand=pd.DataFrame(index=range(0,8760),columns=['Elec','Heat'])     
    ZerDemand.loc[:,'Elec']=ElecDemand.loc[:,str(Group)]
    ZerDemand.loc[:,'Heat']=HeatDemand.loc[:,str(Group)]
    RPotentials=pd.DataFrame(index=range(0,8760),columns=['PV','ST','Hydro'])
    RPotentials.loc[:,'PV']=totalPV.loc[:,str(Group)]
    RPotentials.loc[:,'ST']=totalST.loc[:,str(Group)]
    RPotentials.loc[:,'Hydro']=Hydro.loc[:,'Hydro']
    MaxRoof=RoofArea.loc[Group-1,'RoofArea']
    MaxEvap=DGSHP.loc[Group-1,'GPotential']
    Demand=ZerDemand.as_matrix()
    RPotential=RPotentials.as_matrix()
    return[MaxRoof,MaxEvap,Demand,RPotential]

In [41]:
HPTech = [
    ("GSHP",8889.2,2178,0.11,2.8,15.0,5.5,0,1,1200),
    ("ASHP",11013,1332.5,0.11,2.8,15.0,5.5,0,1,500)]
BTech =[
    ("Gas-boiler",12658,605.7,0.9, 25.0,3.7,0,3,1,10000),
    ("Oil-boiler",17267,463.63,0.85,25.0,3.7,0,5,1,10000),
    ("Biomass-boiler",27404,1024.2, 0.75, 20.0,0,0.0054,4,1,1000)]
CHPTech = [("PEMFC",53158,5748.2,0.5848,0.0004,0.15,1.9,11.5,0.025,2,0,1000,0.5852),
    ("PEMEC",23729,1975.5,0.5485,-0.0006,0.15,1.0894,9,0.025,0,2,1000,0.5452)]
RTech = [("PV",1075.7,267.15,0,0.034,30,6,0),
    ("ST",5608.5,1671.5,9,0,30,6,1)]
CompTech=[("H2Comp",300,0.04,15,2,0)]
StorageTech=[("Battery",2000,943.15,10,25,0.92,0.92,0.001,0.5,0.5,0,100000),
    ("TES",1685,12.604,0,17,0.9,1,0.01,0.25,0.25,1,50000),
    ("Hydrogen",24,17.1,0,20,1,1,0,1,1,2,100000000000)]#prices in KG!

FuelProperties=[("Elec",0.198,0.029),
                ("Heat",0,0),
                ("Hydrogen",0,0),
                ("NG",0.067,0.198),
                ("Biomass",0.075,0),
                ("Oil",0.062,0.270),
                ("Solar",0,0),
                ("Water",0,0)]
                
#CHPtech=[    ("MGT", 900, 0.3, 10.0,0,0.01)]
#0=Elec, 1=Heat, 2=Hydrogen, 3=Natural gas,4=biomass,5=oil
discount=0.06  
#Demand=[(1000, 500, 600, 800, 1600),
 #       (1400, 1700, 900, 200, 903)]
Horizon=range(0,len(Demand))
hps=range(0,len(HPTech))
boil=range(0,len(BTech))
renew=range(0,len(RTech))
chp=range(0,len(CHPTech))
stor=range(0,len(StorageTech))
comp=range(0,len(CompTech))
DemandCarrier=range(0,2)
fuel=range(0,len(FuelProperties))
techr=namedtuple("RTech",["name","fixed_cost","var_cost","OMF","OMV","Lifetime","ECin","ECout"])
techhp = namedtuple("HPTech", ["name", "fixed_cost","var_cost", "eff_slope","eff_int", "Lifetime","OMF","ECin","ECout","MaxCap"])
techb = namedtuple("BTech", ["name", "fixed_cost","var_cost", "eff", "Lifetime","OMF","OMV","ECin","ECout","MaxCap"])
techchp = namedtuple("CHPTech",["name","fixed_cost","var_cost","eff_slope","eff_int","Lower","Upper","Lifetime","OMV","ECin","ECout","MaxCap","OutFactor"])
carrier= namedtuple("FuelProperties",["name","FuelCost","FuelCO2"])
techcomp = namedtuple("CompTech",["name","unit_cost","eff","Lifetime","ECin","ECout"])
strge=namedtuple("StorageTech",["name","fixed_cost","var_cost","OMF","Lifetime","ChrgEff","DschEff","Decay","MaxChrg","MaxDsch","Carrier","MaxCap"])
TechHP = [techhp(*p) for p in HPTech]
TechB = [techb(*b) for b in BTech]
TechRenew = [techr(*r) for r in RTech]
TechCHP = [techchp(*d) for d in CHPTech]
FuelProp = [carrier(*f) for f in FuelProperties]
TechStor = [strge(*s) for s in StorageTech]
CompTech = [techcomp(*c) for  c in CompTech]
AnnuityHP=[None] * len(HPTech)
AnnuityB=[None] * len(BTech)
AnnuityCHP=[None] * len(CHPTech)
AnnuityR=[None]*len(RTech)
AnnuityS=[None]*len(StorageTech)
AnnuityComp=[None]*len(CompTech)
COP=pd.DataFrame(index=Horizon,columns=['GSHP','ASHP'])
for h in Horizon:
    COP.loc[h,'GSHP']=Temp.loc[h,'GroundTemp']*TechHP[0].eff_slope+TechHP[0].eff_int
    COP.loc[h,'ASHP']=Temp.loc[h,'AirTemp']*TechHP[1].eff_slope+TechHP[1].eff_int
for p in hps:
    AnnuityHP[p]=discount/(1-(1/((1+discount)**TechHP[p].Lifetime)))
for b in boil:
    AnnuityB[b]=discount/(1-(1/((1+discount)**TechB[b].Lifetime)))
for d in chp:
    AnnuityCHP[d]=discount/(1-(1/((1+discount)**TechCHP[d].Lifetime)))
for r in renew:
    AnnuityR[r]=discount/(1-(1/((1+discount)**TechRenew[r].Lifetime)))
for s in stor:
    AnnuityS[s]=discount/(1-(1/((1+discount)**TechStor[s].Lifetime)))
for c in comp:
    AnnuityComp[c]=discount/(1-(1/((1+discount)**CompTech[c].Lifetime)))
COPHP=COP.as_matrix()

In [46]:
def build_model(MaxEvap,MaxRoof,Demand,RPotential,hps,boil,renew,chp,TechHP,TechB,TechRenew,TechCHP,FuelProp,TechStor,CompTech,AnnuityHP,AnnuityB,AnnuityCHP,AnnuityR,AnnuityS,AnnuityComp,COPHP):
    mdl = Model(name='BuildingHub')
    Horizon = range(0, len(Demand))
    # Sizing decision variables
    mdl.SizeHP = mdl.integer_var_list(hps,lb=0,name='SizeHP')
    mdl.SizeBoiler = mdl.integer_var_list(boil,lb=0,name='SizeBoiler')
    mdl.SizeCHP = mdl.integer_var_list(chp,lb=0,name='SizeCHP')
    mdl.SizeR = mdl.integer_var_list(renew,lb=0,name='SizeR')
    mdl.SizeStor = mdl.integer_var_list(stor,lb=0,name='SizeStor')
    mdl.SizeComp = mdl.integer_var(lb=0,name='SizeComp')

    #Installation Binaries
    mdl.InstBoiler=mdl.binary_var_list(boil,name='InstBoiler')
    mdl.InstCHP=mdl.binary_var_list(chp,name='InstCHP')
    mdl.InstHP=mdl.binary_var_list(hps,name='InstHP')
    mdl.InstR=mdl.binary_var_list(renew,name='InstR')
    mdl.InstStor=mdl.binary_var_list(stor,name='InstStor')
    mdl.add_constraints(mdl.SizeBoiler[b]<=TechB[b].MaxCap*mdl.InstBoiler[b] for b in boil)
    mdl.add_constraints(mdl.SizeCHP[d]<=TechCHP[d].MaxCap*mdl.InstCHP[d] for d in chp)
    mdl.add_constraints(mdl.SizeHP[p]<=TechHP[p].MaxCap*mdl.InstHP[p] for p in hps)
    mdl.add_constraints(mdl.SizeStor[s]<=TechStor[s].MaxCap*mdl.InstStor[s] for s in stor)
    mdl.add_constraint(mdl.SizeR[0]<=MaxRoof*mdl.InstR[0])
    mdl.add_constraint(mdl.SizeR[1]<=MaxRoof*mdl.InstR[1])
    #mdl.add_constraint(mdl.SizeR[2]==2300)

    mdl.add_constraint(mdl.sum(mdl.InstBoiler[b] for b in boil)==1)
    mdl.add_constraint(mdl.sum(mdl.InstHP[p] for p in hps)==1)
    #Aux variable

    #mdl.yBoiler=mdl.binary_var_matrix(boil,Horizon,name='yBoiler')
    mdl.yCHP=mdl.binary_var_matrix(chp,Horizon,name='yCHP')
    #mdl.sdCHP=mdl.binary_var_matrix(chp,Horizon,name='sdCHP')
    #mdl.suCHP=mdl.binary_var_matrix(chp,Horizon,name='suCHP')
    mdl.DHPump=mdl.continuous_var_list(Horizon,lb=0,name='DHPump')
    mdl.AuxCHP=mdl.continuous_var_matrix(chp,Horizon,lb=0,name='AuxCHP')
    #mdl.AuxCHPsd=mdl.continuous_var_matrix(chp,Horizon,lb=0,name='AuxCHPsd')
    #mdl.AuxCHPsu=mdl.continuous_var_matrix(chp,Horizon,lb=0,name='AuxCHPsu')
    #Aux constraints

    mdl.add_constraints(mdl.AuxCHP[d,h]<=mdl.yCHP[d,h]*TechCHP[d].MaxCap for d in chp for h in Horizon)
    mdl.add_constraints(mdl.AuxCHP[d,h]<=mdl.SizeCHP[d] for d in chp for h in Horizon)
    mdl.add_constraints(mdl.AuxCHP[d,h]>=mdl.SizeCHP[d]-TechCHP[d].MaxCap*(1-mdl.yCHP[d,h]) for d in chp for h in Horizon)

    #mdl.add_constraints(mdl.AuxCHPsd[d,h]<=mdl.sdCHP[d,h]*TechCHP[d].MaxCap for h in Horizon for d in chp)
    #mdl.add_constraints(mdl.AuxCHPsd[d,h]<=mdl.SizeCHP[d] for h in Horizon for d in chp)
    #mdl.add_constraints(mdl.AuxCHPsd[d,h]>=mdl.SizeCHP[d]-TechCHP[d].MaxCap*(1-mdl.sdCHP[d,h]) for h in Horizon for d in chp)
    #mdl.add_constraints(mdl.AuxCHPsu[d,h]<=mdl.suCHP[d,h]*TechCHP[d].MaxCap for h in Horizon)
    #mdl.add_constraints(mdl.AuxCHPsu[d,h]<=mdl.SizeCHP[d] for h in Horizon for d in chp)
    #mdl.add_constraints(mdl.AuxCHPsu[d,h]>=mdl.SizeCHP[d]-TechCHP[d].MaxCap*(1-mdl.suCHP[d,h]) for h in Horizon)
    #Operation decision variables
    mdl.HPOut = mdl.continuous_var_matrix(hps,Horizon,lb=0,name='HPOut')
    mdl.HPIn = mdl.continuous_var_matrix(hps,Horizon,lb=0,name='HPIn')
    mdl.BoilOut=mdl.continuous_var_matrix(boil,Horizon,lb=0,name='BoilOut')
    mdl.BoilIn=mdl.continuous_var_matrix(boil,Horizon,lb=0,name='BoilIn')
    mdl.CompElec = mdl.continuous_var_list(Horizon,lb=0,name='CompElec')
    mdl.CHPIn = mdl.continuous_var_matrix(chp,Horizon,lb=0,name='CHPIn')
    mdl.CHPOut = mdl.continuous_var_matrix(chp,Horizon,lb=0,name='CHPOut')
    mdl.CHPheat=mdl.continuous_var_list(Horizon,lb=0,name='CHPheat')
    mdl.ROut = mdl.continuous_var_matrix(renew,Horizon,lb=0,name='ROut')
    mdl.ElecIn = mdl.continuous_var_list(Horizon,lb=0,name='ElecIn')
    mdl.ElecOut = mdl.continuous_var_list(Horizon,lb=0,name='ElecOut')
    mdl.HeatDump = mdl.continuous_var_list(Horizon,lb=0,name='HeatDump')
    mdl.NGIn = mdl.continuous_var_list(Horizon,lb=0,name='NGIn')
    mdl.StorCharge = mdl.continuous_var_matrix(stor,Horizon,lb=0,name='StorCharge')
    mdl.StorDischarge = mdl.continuous_var_matrix(stor,Horizon,lb=0,name='StorDischarge')
    mdl.StorLevel = mdl.continuous_var_matrix(stor,Horizon,lb=0,name='StorLevel')
    mdl.DI = mdl.continuous_var_list(Horizon,lb=0,name='DI')
    mdl.add_constraints(mdl.DHPump[h]==0.04*Demand[h,1] for h in Horizon)
    ##COST DECISION VARIABLES
    #Capital costs

    #capcost
    mdl.HPCapCost = mdl.continuous_var_list(hps,lb=0,name='HPCapCost')
    mdl.BoilCapCost = mdl.continuous_var_list(boil,lb=0,name='BoilCapCost')
    mdl.CompCapCost = mdl.continuous_var(lb=0,name='CompCapCost')
    mdl.StorCapCost = mdl.continuous_var_list(stor,lb=0,name='StorCapCost')
    mdl.CHPCapCost = mdl.continuous_var_list(chp,lb=0,name='CHPCapCost')
    mdl.RCapCost = mdl.continuous_var_list(renew,lb=0,name='RCapCost')
    #O&M cost
    mdl.OMFCostHP = mdl.continuous_var_list(hps,lb=0,name='OMFCostHP')
    mdl.OMFCostBoil = mdl.continuous_var_list(boil,lb=0,name='OMFCostBoil')
    mdl.OMFCostStor = mdl.continuous_var_list(stor,lb=0,name='OMFCostStor')
    mdl.OMVCostBoil = mdl.continuous_var_list(boil,lb=0,name='OMVCostBoil')
    mdl.OMVCostCHP = mdl.continuous_var_list(chp,lb=0,name='OMVCostCHP')
    mdl.OMFCostR = mdl.continuous_var_list(renew,lb=0,name='OMFCostR')
    mdl.OMVCostR = mdl.continuous_var_list(renew,lb=0,name='OMVCostR')
    #Fuel costs
    mdl.NGCost = mdl.continuous_var(lb=0,name='NGCost')
    mdl.OilCost = mdl.continuous_var(lb=0,name='OilCost')
    mdl.WoodCost = mdl.continuous_var(lb=0,name='WoodCost')
    mdl.ElecCost = mdl.continuous_var(lb=0,name='ElecCost')
    mdl.FITProfit = mdl.continuous_var(lb=0,name='FITProfit')
    #Cost objective
    mdl.TotalCost = mdl.continuous_var(lb=0,name='TotalCost')
    #CO2 decision variables
    mdl.ElecCO2 = mdl.continuous_var(lb=0,name='ElecCO2')
    mdl.NGCO2 = mdl.continuous_var(lb=0,name='NGCO2')
    mdl.OilCO2 = mdl.continuous_var(lb=0,name='OilCO2')
    mdl.TotalCO2 = mdl.continuous_var(lb=0,name='TotalCO2')

    #MaxCapConstraint
    mdl.add_constraints(TechHP[p].MaxCap>=mdl.SizeHP[p] for p in hps)
    mdl.add_constraints(TechCHP[d].MaxCap>=mdl.SizeCHP[d] for d in chp)
    mdl.add_constraints(TechB[b].MaxCap>=mdl.SizeBoiler[b] for b in boil)
    #Sizing constraint
    mdl.add_constraints(mdl.SizeHP[p] >=mdl.HPOut[p,h] for p in hps for h in Horizon)
    mdl.add_constraints(mdl.SizeBoiler[b]>=mdl.BoilOut[b,h] for b in boil for h in Horizon)
    mdl.add_constraints(mdl.AuxCHP[d,h]>=mdl.CHPIn[d,h]*TechCHP[d].Upper for d in chp for h in Horizon)
    mdl.add_constraints(mdl.AuxCHP[d,h]*TechCHP[d].Lower<=mdl.CHPIn[d,h] for d in chp for h in Horizon)
    mdl.add_constraints(mdl.SizeStor[s]>=mdl.StorLevel[s,h] for s in stor for h in Horizon)
    mdl.add_constraints(mdl.SizeComp>=mdl.CompElec[h] for h in Horizon)
    #Simple eff conv
    mdl.add_constraints(mdl.HPOut[p,h]==mdl.HPIn[p,h]*COPHP[h,p] for p in hps for h in Horizon)
    mdl.add_constraints(mdl.ROut[0,h]==RPotential[h,0]*mdl.SizeR[0] for h in Horizon) #
    mdl.add_constraints(mdl.ROut[1,h]==RPotential[h,1]*mdl.SizeR[1] for h in Horizon) #
    #mdl.add_constraints(mdl.ROut[2,h]==RPotential[h,2]*mdl.SizeR[2]/2300 for h in Horizon)
    mdl.add_constraint(mdl.sum(mdl.SizeR[r] for r in renew if TechRenew[r].ECin==6)<=MaxRoof)

    #CHP eff
    mdl.add_constraints(mdl.CHPOut[d,h]==mdl.CHPIn[d,h]*TechCHP[d].eff_slope+mdl.AuxCHP[d,h]*TechCHP[d].eff_int for d in chp for h in Horizon)
    mdl.add_constraints(mdl.CHPOut[d,h]<=mdl.AuxCHP[d,h]*TechCHP[d].OutFactor for d in chp for h in Horizon)
    mdl.add_constraints(mdl.CHPOut[0,h]*(1.71-1)==mdl.CHPheat[h] for h in Horizon)
    #Boiler eff
    mdl.add_constraints(mdl.BoilOut[b,h]==mdl.BoilIn[b,h]*TechB[b].eff for b in boil for h in Horizon)
    #Comp elec
    mdl.add_constraints(mdl.CompElec[h]==mdl.StorCharge[2,h]*CompTech[p].eff for p in comp for h in Horizon)
    #HP constraint
    mdl.add_constraints(mdl.HPOut[0,h]-mdl.HPIn[0,h]<=MaxEvap for h in Horizon)
    #Energy balances
    mdl.add_constraints(mdl.sum(mdl.HPOut[p,h] for p in hps)+mdl.sum(mdl.BoilOut[b,h] for b in boil)+mdl.CHPheat[h]+mdl.sum(mdl.ROut[r,h] for r in renew if TechRenew[r].ECout==1)+mdl.sum(mdl.StorDischarge[s,h] for s in stor if TechStor[s].Carrier==1)==Demand[h,1]+mdl.sum(mdl.StorCharge[s,h] for s in stor if TechStor[s].Carrier==1)+mdl.HeatDump[h] for h in Horizon)
    mdl.add_constraints(mdl.CHPOut[0,h]+mdl.sum(mdl.ROut[r,h] for r in renew if TechRenew[r].ECout==0)+mdl.sum(mdl.StorDischarge[0,h])+mdl.ElecIn[h]==Demand[h,0]+mdl.sum(mdl.HPIn[p,h] for p in hps)+mdl.DHPump[h]+mdl.ElecOut[h]+mdl.CompElec[h]+mdl.sum(mdl.StorCharge[0,h])+mdl.sum(mdl.CHPIn[1,h]) for h in Horizon)
    mdl.add_constraints(mdl.BoilIn[0,h]==mdl.NGIn[h]+mdl.DI[h] for h in Horizon) #assuming no MGT
    mdl.add_constraints(mdl.CHPIn[0,h]+mdl.DI[h]==mdl.StorDischarge[2,h] for h in Horizon) #assuming no direct injection
    mdl.add_constraints(mdl.CHPOut[1,h]==mdl.StorCharge[2,h] for h in Horizon)
    mdl.add_constraints(mdl.DI[h]<=0.03606*mdl.BoilIn[0,h] for h in Horizon)
    #Storage Constraints
    for h in Horizon:
        if h==0:
            mdl.add_constraints(mdl.StorLevel[s,h]==mdl.StorLevel[s,8759] for s in stor)
            mdl.add_constraints(mdl.StorDischarge[s,h]==0 for s in stor)
            mdl.add_constraints(mdl.StorCharge[s,h]==0 for s in stor)
        else:
            mdl.add_constraints(mdl.StorLevel[s,h]==mdl.StorLevel[s,h-1]*(1-TechStor[s].Decay)+mdl.StorCharge[s,h]*TechStor[s].ChrgEff-mdl.StorDischarge[s,h]*(1/TechStor[s].DschEff) for s in stor)
    mdl.add_constraints(mdl.StorCharge[s,h]<=TechStor[s].MaxChrg*mdl.SizeStor[s] for s in stor for h in Horizon)
    mdl.add_constraints(mdl.StorDischarge[s,h]<=TechStor[s].MaxDsch*mdl.SizeStor[s] for s in stor for h in Horizon)
    #FIT Constraints
    mdl.add_constraints(mdl.ROut[0,h]>=mdl.ElecOut[h] for h in Horizon)
    #mdl.add_constraints((1-y_buysell[h])*10000>=ElecIn[h] for h in Horizon)
    ##ramp constraints
    #for h in Horizon:
     #   if h==0:
            #mdl.add_constraints(mdl.sdCHP[d,h]==0 for d in chp)
            #mdl.add_constraints(mdl.suCHP[d,h]==0 for d in chp)
        #else:
            #mdl.add_constraints(mdl.suCHP[d,h]-mdl.sdCHP[d,h]==mdl.yCHP[d,h]-mdl.yCHP[d,h-1] for d in chp)
    #for h in Horizon:
        #if h==(len(Demand)-1):
            #mdl.add_constraints(mdl.CHPIn[d,h]>=(0.18*(mdl.AuxCHP[d,h]-mdl.AuxCHPsu[d,h]))+(0.25*0.18*mdl.AuxCHPsu[d,h]) for d in chp)
            #mdl.add_constraints(mdl.CHPIn[d,h]<=(mdl.AuxCHP[d,h]-mdl.AuxCHPsu[d,h])+((1-0.18)*mdl.AuxCHPsu[d,h]*(1-0.25)+0.18*mdl.AuxCHPsu[d,h]) for d in chp)
            #mdl.add_constraints(mdl.AuxCHPsd[d,h]==0 for d in chp)
        #else:
            #mdl.add_constraints(mdl.CHPIn[d,h]>=(0.18*(mdl.AuxCHP[d,h]-mdl.AuxCHPsd[d,h+1]-mdl.AuxCHPsu[d,h]))+(0.25*0.18*mdl.AuxCHPsu[d,h]) for d in chp)#ramp up min
            #mdl.add_constraints(mdl.CHPIn[d,h]<=mdl.AuxCHP[d,h]-mdl.AuxCHPsu[d,h]+((1-0.18)*mdl.AuxCHPsu[d,h]*(1-0.25)+0.18*mdl.AuxCHPsu[d,h]) for d in chp)#ramp up max
            #mdl.add_constraints(mdl.CHPIn[d,h]<=(mdl.AuxCHP[d,h]-mdl.AuxCHPsd[d,h+1])+((1-0.18)*mdl.AuxCHPsd[d,h+1]*(1-0.25)+0.18*mdl.AuxCHPsd[d,h+1])for d in chp)
            #mdl.add_constraints(mdl.AuxCHP[d,h]-mdl.AuxCHPsd[d,h+1]-mdl.AuxCHPsu[d,h]>=0 for d in chp)
    #calculate costs

    #mdl.add_constraint(mdl.CHPIn[1,4000]>=100)
    #mdl.add_constraint(mdl.SizeCHP[1]>=100)#del
    #mdl.add_constraint(mdl.SizeCHP[0]>=100)#del
    #mdl.add_constraint(mdl.SizeStor[0]==0)#del
    #mdl.add_constraint(mdl.SizeStor[1]==0)#del
    #mdl.add_constraint(mdl.SizeStor[2]>=10000)
    mdl.add_constraints(mdl.HPCapCost[p]==AnnuityHP[p]*(TechHP[p].fixed_cost*mdl.InstHP[p]+TechHP[p].var_cost*mdl.SizeHP[p]) for p in hps)
    mdl.add_constraints(mdl.BoilCapCost[b]==AnnuityB[b]*(TechB[b].fixed_cost*mdl.InstBoiler[b]+TechB[b].var_cost*mdl.SizeBoiler[b]) for b in boil)
    mdl.add_constraints(mdl.StorCapCost[s]==AnnuityS[s]*(TechStor[s].fixed_cost*mdl.InstStor[s]+mdl.SizeStor[s]*TechStor[s].var_cost) for s in stor)
    mdl.add_constraints(mdl.CompCapCost==AnnuityComp[p]*CompTech[p].unit_cost*mdl.SizeComp for p in comp)
    mdl.add_constraints(mdl.CHPCapCost[d]==AnnuityCHP[d]*(TechCHP[d].fixed_cost*mdl.InstCHP[d]+TechCHP[d].var_cost*mdl.SizeCHP[d]) for d in chp)
    mdl.add_constraints(mdl.RCapCost[r]==AnnuityR[r]*(TechRenew[r].fixed_cost*mdl.InstR[r]+mdl.SizeR[r]*TechRenew[r].var_cost) for r in renew)
    mdl.add_constraints(mdl.OMFCostHP[p]==TechHP[p].OMF*mdl.SizeHP[p] for p in hps)

    mdl.add_constraints(mdl.OMFCostStor[s]==TechStor[s].OMF*mdl.SizeStor[s] for s in stor)
    mdl.add_constraints(mdl.OMFCostBoil[b]==TechB[b].OMF*mdl.SizeBoiler[b] for b in boil)
    mdl.add_constraints(mdl.OMVCostBoil[b]==mdl.sum(mdl.BoilOut[b,h] for h in Horizon)*TechB[b].OMV for b in boil)
    mdl.add_constraints(mdl.OMVCostCHP[d]==mdl.sum(mdl.CHPOut[d,h]for h in Horizon)*TechCHP[d].OMV for d in chp)
    mdl.add_constraints(mdl.OMFCostR[r]==mdl.SizeR[r]*TechRenew[r].OMF for r in renew)
    mdl.add_constraints(mdl.OMVCostR[r]==mdl.sum(mdl.ROut[r,h] for h in Horizon)*TechRenew[r].OMV for r in renew)
    mdl.add_constraint(mdl.NGCost==0.067*mdl.sum(mdl.NGIn[h] for h in Horizon))
    mdl.add_constraint(mdl.NGCO2==0.20*mdl.sum(mdl.NGIn[h] for h in Horizon))
    mdl.add_constraint(mdl.OilCost==0.062*mdl.sum(mdl.BoilOut[1,h] for h in Horizon))
    mdl.add_constraint(mdl.WoodCost==0.075*mdl.sum(mdl.BoilOut[2,h] for h in Horizon))
    mdl.add_constraint(mdl.OilCO2==0.27*mdl.sum(mdl.BoilOut[1,h] for h in Horizon))
    mdl.add_constraint(mdl.ElecCost==0.198*mdl.sum(mdl.ElecIn[h] for h in Horizon))
    mdl.add_constraint(mdl.FITProfit==0.12*mdl.sum(mdl.ElecOut[h] for h in Horizon)) #assuming to non-renewable eletricity production
    mdl.add_constraint(mdl.ElecCO2==mdl.sum(mdl.ElecIn[h] for h in Horizon)*0.09)
    mdl.add_constraint(mdl.TotalCost==mdl.sum(mdl.HPCapCost[p] for p in hps)+mdl.sum(mdl.BoilCapCost[b] for b in boil)+mdl.WoodCost+mdl.sum(mdl.CHPCapCost[d] for d in chp)+mdl.sum(mdl.RCapCost[r] for r in renew)+mdl.CompCapCost+mdl.sum(mdl.StorCapCost[s] for s in stor)+mdl.sum(mdl.OMFCostHP[p] for p in hps)+mdl.sum(mdl.OMFCostBoil[b] for b in boil)+mdl.sum(mdl.OMFCostR[r] for r in renew)+mdl.sum(mdl.OMFCostStor[s] for s in stor)+mdl.sum(mdl.OMVCostCHP[d] for d in chp)+mdl.sum(mdl.OMVCostR[r] for r in renew)+mdl.sum(mdl.OMVCostBoil[b] for b in boil)-mdl.FITProfit+mdl.NGCost+mdl.ElecCost+mdl.OilCost)
    mdl.add_constraint(mdl.TotalCO2==mdl.ElecCO2+mdl.NGCO2+mdl.OilCO2)
    #mdl.add_constraint(mdl.TotalCO2<=EpConst)
    mdl.add_constraint(mdl.SizeBoiler[2]==0)
    mdl.print_information()
    # Minimize cost
    return mdl

In [48]:
def cost_min(mdl):
    mdl.minimize(mdl.TotalCost)
    return mdl

In [49]:
def co2_min(mdl):
    mdl.minimize(mdl.TotalCost*0.000001+mdl.TotalCO2)
    return mdl

In [ ]:
#not for building level!
for i in range(1,19):
    #Econstraint=Econst.loc[i-1,'1']
    Costname='NCost_'+str(i)
    CO2name='NCO2_'+str(i)
    MaxRoof,MaxEvap,Demand,RPotential=load_data(i,ElecDemand,HeatDemand,totalPV,totalST,TotalRoofArea,DGSHP)
    mm=build_model(MaxEvap,MaxRoof,Demand,RPotential,hps,boil,renew,chp,TechHP,TechB,TechRenew,TechCHP,FuelProp,TechStor,CompTech,AnnuityHP,AnnuityB,AnnuityCHP,AnnuityR,AnnuityS,AnnuityComp,COPHP)
    mc=cost_min(mm)
    mc.export_as_lp(basename=Costname,path='E:\Master2')

In [37]:
HeatOut=pd.DataFrame(index=Horizon,columns=['GSHP','ASHP','Gas_boiler','Oil_boiler','ST','PEMFC'])
ElecOuts=pd.DataFrame(index=Horizon,columns=['ElecIn','PV','PEMFC'])
Extras=pd.DataFrame(index=Horizon,columns=['NGin','ElecOut','HeatDump','PEMEC-H2'])
AuxBoilers=pd.DataFrame(index=Horizon,columns=['AuxGasBoiler','AuxOilBoiler','yGasBoiler','yOilBoiler'])
AuxCHPs=pd.DataFrame(index=Horizon,columns=['AuxPEMFC-elec','AuxPEMEC','yPEMFC-elec','yPEMEC'])
Charge=pd.DataFrame(index=Horizon,columns=['Bat','TES','Hydrogen'])
Discharge=pd.DataFrame(index=Horizon,columns=['Bat','TES','Hydrogen'])
Level=pd.DataFrame(index=Horizon,columns=['Bat','TES','Hydrogen'])
for h in Horizon:
    HeatOut.loc[h,'Gas_boiler']=mdl.BoilOut[0,h].solution_value
    HeatOut.loc[h,'GSHP']=mdl.HPOut[0,h].solution_value
    HeatOut.loc[h,'ASHP']=mdl.HPOut[1,h].solution_value
    HeatOut.loc[h,'Oil_boiler']=mdl.BoilOut[1,h].solution_value
    HeatOut.loc[h,'PEMFC']=mdl.CHPheat[h].solution_value
    HeatOut.loc[h,'ST']=mdl.ROut[1,h].solution_value
    ElecOuts.loc[h,'PV']=mdl.ROut[0,h].solution_value
    ElecOuts.loc[h,'ElecIn']=mdl.ElecIn[h].solution_value
    ElecOuts.loc[h,'PEMFC']=mdl.CHPOut[0,h].solution_value
    Extras.loc[h,'NGin']=mdl.NGIn[h].solution_value
    Extras.loc[h,'PEMEC-H2']=mdl.CHPOut[1,h].solution_value
    Extras.loc[h,'ElecOut']=mdl.ElecOut[h].solution_value
    Extras.loc[h,'HeatDump']=mdl.HeatDump[h].solution_value
    AuxCHPs.loc[h,'AuxPEMFC-elec']=mdl.AuxCHP[0,h].solution_value
    AuxCHPs.loc[h,'AuxPEMEC']=mdl.AuxCHP[1,h].solution_value
    AuxCHPs.loc[h,'yPEMFC-elec']=mdl.yCHP[0,h].solution_value
    AuxCHPs.loc[h,'yPEMEC']=mdl.yCHP[1,h].solution_value
    Charge.loc[h,'Bat']=mdl.StorCharge[0,h].solution_value
    Discharge.loc[h,'Bat']=mdl.StorDischarge[0,h].solution_value
    Level.loc[h,'Bat']=mdl.StorLevel[0,h].solution_value
    Charge.loc[h,'TES']=mdl.StorCharge[1,h].solution_value
    Discharge.loc[h,'TES']=mdl.StorDischarge[1,h].solution_value
    Level.loc[h,'TES']=mdl.StorLevel[1,h].solution_value
    Charge.loc[h,'Hydrogen']=mdl.StorCharge[2,h].solution_value
    Discharge.loc[h,'Hydrogen']=mdl.StorDischarge[2,h].solution_value
    Level.loc[h,'Hydrogen']=mdl.StorLevel[2,h].solution_value

In [38]:
EnIn=pd.DataFrame(index=Horizon,columns=['GSHP','ASHP','Gas_boiler','Oil_boiler','Comp-elec','PEMFC-H2','PEMEC-Elec','ElecIn'])
for h in Horizon:
    EnIn.loc[h,'Gas_boiler']=mdl.BoilIn[0,h].solution_value
    EnIn.loc[h,'GSHP']=mdl.HPIn[0,h].solution_value
    EnIn.loc[h,'ASHP']=mdl.HPIn[1,h].solution_value
    EnIn.loc[h,'Comp-elec']=mdl.CompElec[h].solution_value
    EnIn.loc[h,'Oil_boiler']=mdl.BoilIn[1,h].solution_value
    EnIn.loc[h,'PEMFC-H2']=mdl.CHPIn[0,h].solution_value
    EnIn.loc[h,'PEMEC-Elec']=mdl.CHPIn[1,h].solution_value
    EnIn.loc[h,'ElecIn']=mdl.ElecIn[h].solution_value

In [39]:
ctec=['GSHP','ASHP','Gas_boiler','Oil_boiler','Comp-elec']
ConvSize=pd.DataFrame(index=ctec,columns=['Size','CapCost','OMF','OMV','ProdOut','EnIn'])
for h in hps:
    ConvSize.loc[ctec[h],'Size']=mdl.SizeHP[h].solution_value
    ConvSize.loc[ctec[h],'CapCost']=mdl.HPCapCost[h].solution_value
    ConvSize.loc[ctec[h],'OMF']=mdl.OMFCostHP[h].solution_value
    ConvSize.loc[ctec[h],'OMV']=0
    ConvSize.loc[ctec[h],'ProdOut']=HeatOut[ctec[h]].sum()
    ConvSize.loc[ctec[h],'EnIn']=EnIn[ctec[h]].sum()
for b in boil:
    print(ctec[b+2])
    ConvSize.loc[ctec[b+2],'Size']=mdl.SizeBoiler[b].solution_value
    ConvSize.loc[ctec[b+2],'CapCost']=mdl.BoilCapCost[b].solution_value
    ConvSize.loc[ctec[b+2],'OMF']=mdl.OMFCostBoil[b].solution_value
    ConvSize.loc[ctec[b+2],'OMV']=mdl.OMVCostBoil[b].solution_value
    ConvSize.loc[ctec[b+2],'ProdOut']=HeatOut[ctec[b+2]].sum()
    ConvSize.loc[ctec[b+2],'EnIn']=EnIn[ctec[b+2]].sum()
ConvSize.loc['Comp-elec','ProdOut']=0
ConvSize.loc['Comp-elec','EnIn']=EnIn['Comp-elec'].sum()
ConvSize

Gas_boiler
Oil_boiler
Comp-elec


KeyError: 'Comp-elec'

In [ ]:
dtec=['PEMFC','PEMEC']
CHPSize=pd.DataFrame(index=dtec,columns=['Size','CapCost','OMF','OMV','Prod','EnIn','Heat'])
for d in chp:
    CHPSize.loc[dtec[d],'Size']=mdl.SizeCHP[d].solution_value
    CHPSize.loc[dtec[d],'CapCost']=mdl.CHPCapCost[d].solution_value
    CHPSize.loc[dtec[d],'OMF']=0
    CHPSize.loc[dtec[d],'OMV']=mdl.OMVCostCHP[d].solution_value
CHPSize.loc['PEMFC','Prod']=ElecOuts['PEMFC'].sum()
CHPSize.loc['PEMFC','Heat']=HeatOut['PEMFC'].sum()
CHPSize.loc['PEMEC','Prod']=Extras['PEMEC-H2'].sum()
CHPSize.loc['PEMEC','EnIn']=EnIn['PEMEC-Elec'].sum()
CHPSize.loc['PEMFC','EnIn']=EnIn['PEMFC-H2'].sum()
CHPSize

In [ ]:
rtec=['PV','ST']
RenewSize=pd.DataFrame(index=['PV','ST'],columns=['Size','CapCost','OMF','OMV','ProdOut'])
for r in renew:
    RenewSize.loc[rtec[r],'Size']=mdl.SizeR[r].solution_value
    RenewSize.loc[rtec[r],'CapCost']=mdl.RCapCost[r].solution_value
    RenewSize.loc[rtec[r],'OMF']=mdl.OMFCostR[r].solution_value
    RenewSize.loc[rtec[r],'OMV']=mdl.OMVCostR[r].solution_value
RenewSize.loc[rtec[0],'ProdOut']=ElecOuts[rtec[0]].sum()
RenewSize.loc[rtec[1],'ProdOut']=HeatOut[rtec[1]].sum()
RenewSize

In [ ]:
stec=['Bat','TES','Hydrogen']
StorSize=pd.DataFrame(index=stec,columns=['Size','CapCost','OMF','EnergyIn','EnergyOut'])

for s in stor:
    StorSize.loc[stec[s],'Size']=mdl.SizeStor[s].solution_value
    StorSize.loc[stec[s],'CapCost']=mdl.StorCapCost[s].solution_value
    StorSize.loc[stec[s],'OMF']=mdl.OMFCostStor[s].solution_value
    StorSize.loc[stec[s],'EnergyIn']=Charge[stec[s]].sum()
    StorSize.loc[stec[s],'EnergyOut']=Discharge[stec[s]].sum()
StorSize

In [148]:
Level.sum()

Bat                0
TES                0
Hydrogen    87600000
dtype: float64